<a href="https://www.kaggle.com/code/patimejia/goes16-contrail-rle-encoding-submission?scriptVersionId=136373897" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

def rle_encode(image, foreground_value=1):
    # Run length encoding.
    dots = np.where(image.T.flatten() == foreground_value)[0]
    run_lengths = []
    prev = -2
    for dot in dots:
        if (dot > prev + 1): run_lengths.extend((dot + 1, 0))
        run_lengths[-1] += 1
        prev = dot
    return run_lengths

def rle_to_string(rle):
    # Converts RLE to a string.
    return '-'.join(map(str, rle)) if rle else '-'

def create_mask(data_path, record, num_pixels, shape=(256, 256)):
    # Creates a mask for the given record.
    band_08 = np.load(data_path / 'test' / record / 'band_08.npy').sum(axis=2)
    preds = np.c_[np.unravel_index(np.argpartition(band_08.ravel(), -num_pixels)[-num_pixels:], band_08.shape)]
    mask = np.zeros(shape)
    mask[preds[:, 0], preds[:, 1]] = 1
    return mask

def visualize_mask(mask):
    # Visualizes the mask using matplotlib.
    plt.imshow(mask, cmap='Greys')
    plt.title("Obviously Not Contrails")
    plt.show()

def prepare_submission(submission, record, mask):
    # Prepares and updates the submission DataFrame.
    rle_mask = rle_encode(mask)
    submission.loc[int(record), 'encoded_pixels'] = rle_to_string(rle_mask)

def main(data_path, num_plots=1):
    # Main function.
    data_path = Path(data_path)
    test_records = [entry.name for entry in data_path.glob('test/*') if entry.is_dir()]
    num_pixels = 1000

    submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='record_id')

    for i, record in enumerate(test_records):
        mask = create_mask(data_path, record, num_pixels)
        if i < num_plots:  # Visualize the masks for the first 'num_plots' records
            visualize_mask(mask)
        prepare_submission(submission, record, mask)

    submission.to_csv('submission.csv')

# Directly call main with your data_path and num_plots as arguments
main('/kaggle/input/google-research-identify-contrails-reduce-global-warming', num_plots=2)

# Contrail Detection Submission with Run-Length Encoding (RLE)
- The script imports necessary libraries such as `numpy`, `pandas`, and `pathlib`.
- Defines several functions to perform RLE, create and visualize masks, and prepare the submission `DataFrame`.
- The main function calls these functions and saves the submission in a CSV file.

## What is RLE?
- Run-length encoding (RLE) is a form of lossless data compression.
- It stores runs of data as a single data value and count.
- For example, a string of characters `AAABBCCCC` can be compressed to `3A2B4C`.
- RLE excels at compressing data with runs: numerous consecutive repetitions.

## Handling Empty Predictions
- Empty predictions are handled by the `rle_to_string` function.
- If the input RLE mask is `empty`, the function returns a string containing a single dash (`'-'`).

# Acknowledgement
- This script is inspired by [INVERSION’s notebook](https://www.kaggle.com/code/inversion/contrails-rle-submission) on Kaggle.
- GitHub: To run this code locally, visit [src/utils/rle_encoding_submission.py](https://github.com/patmejia/contrails-vision/blob/main/src/utils/rle_encoding_submission.py).

---
<div style="background-color: #f2f2f2; padding: 10px; border-radius: 5px;">
    <h4 style="text-align: left;">If you found this notebook helpful...</h4>
    <p style="text-align: left;">Please consider giving it an upvote. Your support helps me continue to develop high-quality code and pursue my career as a data analyst/engineer. Feedback is always welcome and appreciated. Thank you for taking the time to read my work!</p>
    <div style="text-align: right;">
          <button style="background-color: #20BEFF; color: white; padding: 12px 20px; border: none; border-radius: 4px; cursor: pointer; font-size: 14px;" onclick="window.location.href='https://www.kaggle.com/code/patimejia/goes16-contrail-rle-encoding-submission#:~:text=goes16_contrail_rle_encoding_submission'"> Up ⇪</button>
</div>
  